# Second model 

This model will try to predict wether pe is present or not on a full image (RGB level)

## Imports

In [1]:
import numpy as np
from balance import *
from torchsummary import summary

In [2]:
import torch
torch.cuda.empty_cache()

print(f'PyTorch version: {torch.__version__}')
print("GPU found :)" if torch.cuda.is_available() else "No GPU :(")

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

PyTorch version: 1.7.1
GPU found :)


# Load datasets

In [3]:
full_image_dataset = torch.load('datasets/full_image_dataset.pt')

## Simple CNN from scratch

In [4]:
class ModelV2(nn.Module):
    
    def __init__(self, input_channels):
        """Simple Convnet with 2 convolution layer + pooling, with 2 fully connected at the end"""
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.fc1 = nn.Linear(32*62*62 , 120)
        self.fc2 = nn.Linear(120, 1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32*62*62) # Flatten
        x = F.relu(self.fc1(x))
        
        # Binary classification
        out = self.sigmoid(self.fc2(x))
        
        return out

In [5]:
model = ModelV2(3).to(device)
summary(model, (3, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 254, 254]             896
         MaxPool2d-2         [-1, 32, 127, 127]               0
            Conv2d-3         [-1, 32, 125, 125]           9,248
         MaxPool2d-4           [-1, 32, 62, 62]               0
            Linear-5                  [-1, 120]      14,761,080
            Linear-6                    [-1, 1]             121
           Sigmoid-7                    [-1, 1]               0
Total params: 14,771,345
Trainable params: 14,771,345
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.75
Forward/backward pass size (MB): 24.44
Params size (MB): 56.35
Estimated Total Size (MB): 81.54
----------------------------------------------------------------


## Split data into train and test

In [6]:
batch_size = 24
ratio=0.2

train_loader, test_loader = train_test_split(full_image_dataset, ratio, batch_size)

## Model Training

In [7]:
num_epoch = 10
step_count = len(train_loader)
loss_function = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(num_epoch):
    for i, sample in enumerate(train_loader):
        
        image = sample['image'].to(device, dtype=torch.float)
        target = sample['target'].to(device, dtype=torch.float)
        target = target.view(target.shape[0], 1)
        
        # Reset gradiant
        optimizer.zero_grad()
        
        # Forward pass
        pred = model(image)
        
        # Compute loss
        loss = loss_function(pred, target)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        del image
        del target
        del pred
        del sample
        
        # Debug
        if((i+1) % 10 == 0):
            print(
                        f"Epoch [{epoch + 1}/{num_epoch}]"
                        f", step [{i + 1}/{step_count}]"
                        f", loss: {loss.item():.4f}"
                    )

Epoch [1/10], step [10/53], loss: 0.6949
Epoch [1/10], step [20/53], loss: 0.6940
Epoch [1/10], step [30/53], loss: 0.6988
Epoch [1/10], step [40/53], loss: 0.6914
Epoch [1/10], step [50/53], loss: 0.6887
Epoch [2/10], step [10/53], loss: 0.6904
Epoch [2/10], step [20/53], loss: 0.6956
Epoch [2/10], step [30/53], loss: 0.6892
Epoch [2/10], step [40/53], loss: 0.6910
Epoch [2/10], step [50/53], loss: 0.6947
Epoch [3/10], step [10/53], loss: 0.6881
Epoch [3/10], step [20/53], loss: 0.6965
Epoch [3/10], step [30/53], loss: 0.6993
Epoch [3/10], step [40/53], loss: 0.6879
Epoch [3/10], step [50/53], loss: 0.6934
Epoch [4/10], step [10/53], loss: 0.6901
Epoch [4/10], step [20/53], loss: 0.6952
Epoch [4/10], step [30/53], loss: 0.6918
Epoch [4/10], step [40/53], loss: 0.6973
Epoch [4/10], step [50/53], loss: 0.6952
Epoch [5/10], step [10/53], loss: 0.6934
Epoch [5/10], step [20/53], loss: 0.6879
Epoch [5/10], step [30/53], loss: 0.6874
Epoch [5/10], step [40/53], loss: 0.6967
Epoch [5/10], st

In [15]:
# Save model 
torch.save(model.state_dict(), './models/model_v2.h5')
del model

NameError: name 'model' is not defined

## Test the model

In [16]:
# Load model for evaluation
trained_model = ModelV2(input_channels=3)
trained_model.load_state_dict(torch.load('./models/model_v2.h5'))
trained_model.to(device)

ModelV2(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=123008, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [25]:
correct = 0
with torch.no_grad():
    for sample in test_loader:
        
        image = sample['image'].to(device, dtype=torch.float)
        target = sample['target'].to(device)
        
        output = trained_model(image)

        pred = 1 if output[0][0] > 0.5 else 0

        correct += (pred == target[0])

In [27]:
total = len(test_loader)
print(f'Accuracy of the network on the {total} test images: {100 * correct / total:.2f}')

Accuracy of the network on the 1254 test images: 48.33
